## CBOW - Continuous Bag of Words - Word2Vec

Uma das variantes do word2vec é o modelo CBOW (Continuous bag of words). <br>
A ideia desse modelo é obter os embeddings implicitamente por meio de uma tarefa que parece não fazer muito sentido a priori. Vamos treinar uma rede neural rasa que, a partir de *palavras de contexto* vai tentar prever a palavra central. <br>
As palavras de contexto são definidas como as que estão numa janela N ao redor da palavra central. Por exemplo, com a frase `eu quero comer morango com batata`, considerando um N=2 e a palavra central `morango` temos as palavras de contexto `[quero, comer, com, batata]`. A rede deve receber as palavras de contexto e prever a palavra central.
Essa tarefa é dita "auto supervisionada" porque os rótulos não são dados por um ser humano, mas são palavras do proprio texto. <br>
Bom, mas nós não alimentamos as strings puras no modelo. A rede é alimentada com uma média dos vetores *one hot* das palavras de contexto e deve prever o *one hot* da palavra central. 
Vetores *one hot* são vetores populados apenas com zeros, com exceção de um elemento que é o 1. Exemplificando: vamos supor que temos um vocabulário de 3 palavras: 'azul', 'rosa' e 'preto', nesse caso, os *one hot* seriam, [1,0,0], [0,1,0] e [0,0,1] para azul', 'rosa' e 'preto', respectivamente. Nesse caso, a média dos três vetores *one-hot* seria [$\frac{1}{3}$, $\frac{1}{3}$, $\frac{1}{3}$].

![](https://media.geeksforgeeks.org/wp-content/uploads/cbow-1.png)

Portanto, ao receber um texto, devemos prepará-lo para que tenhamos features e rótulos para alimentar o modelo. Para isso, devemos, primeiro extrair os períodos do texto, tokenizá-los e tirar caractéres que não sejam de nosso interesse, como pontuação e números. <br>
Uma vez feito isso, conseguimos definir nosso vocabulário e os *one-hots* correspondentes a cada palavra. <br>
Depois, "escaneamos" cada sentença dentro da janela prevista, preparando vetores X, com as médias dos *one hot* das palavras de contexto, associados a um Y, o *one hot* da palavra central. 

In [1]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import re
import json

In [2]:
f = open("biblia-em-txt.txt", "r")
text = f.read()

In [3]:
text[:500]

'BÍBLIA SAGRADA\nTradução: João Ferreira de Almeida\nEdição Revista e Corrigida\n \t\nANTIGO TESTAMENTO\n\nGÊNESIS\n GÊNESIS 1\n1 No princípio criou Deus os céus e a terra.\n2 A terra era sem forma e vazia; e havia trevas sobre a face do abismo, mas o Espírito de Deus pairava sobre a face das águas.\n3 Disse Deus: haja luz. E houve luz.\n4 Viu Deus que a luz era boa; e fez separação entre a luz e as trevas.\n5 E Deus chamou à luz dia, e às trevas noite. E foi a tarde e a manhã, o dia primeiro.\n6 E disse Deus:'

In [4]:
def tokenize_sentece(sent):
    tokenized = word_tokenize(sent)
    tokenized = [token.lower() for token in tokenized if token.isalpha()]
    return tokenized

def tokenize_text(text):
    sentences = sent_tokenize(text)
    tokenized_senteces = [tokenize_sentece(sent) for sent in sentences]
    return tokenized_senteces

In [5]:
sent_tokens = tokenize_text(text)

Os tokens ficam assim:

In [6]:
sent_tokens[0:2]

[['bíblia',
  'sagrada',
  'tradução',
  'joão',
  'ferreira',
  'de',
  'almeida',
  'edição',
  'revista',
  'e',
  'corrigida',
  'antigo',
  'testamento',
  'gênesis',
  'gênesis',
  'no',
  'princípio',
  'criou',
  'deus',
  'os',
  'céus',
  'e',
  'a',
  'terra'],
 ['a',
  'terra',
  'era',
  'sem',
  'forma',
  'e',
  'vazia',
  'e',
  'havia',
  'trevas',
  'sobre',
  'a',
  'face',
  'do',
  'abismo',
  'mas',
  'o',
  'espírito',
  'de',
  'deus',
  'pairava',
  'sobre',
  'a',
  'face',
  'das',
  'águas']]

In [7]:
def make_vocab(sent_tokenized):
    i = 0
    vocab = {}
    for sent in sent_tokenized:
        for token in sent:
            if token in vocab:
                continue
            else:
                vocab[token] = i
                i+=1
    return vocab

In [8]:
vocab = make_vocab(sent_tokens)

In [9]:
V = len(vocab)

In [10]:
def get_windows(words, C):
    i = C
    while i < len(words) - C:
        center_word = words[i]
        context_words = words[(i - C):i] + words[(i+1):(i+C+1)]
        yield context_words, center_word
        i += 1

In [12]:
def make_dataset(sent_tokens, vocab, V, C):
    windows_and_target = []
    for sentence in sent_tokens:
        for x, y in get_windows(sentence,C):
            windows_and_target.append((x,y))
        
    return windows_and_target

In [13]:
dataset = make_dataset(sent_tokens, vocab, V, C=2)

Temos o dataset na seguinte estrutura:

In [15]:
idx = np.random.randint(low=0,high=len(dataset), size=3)
for i in idx:
    print(dataset[i])

(['nos', 'céus', 'na', 'terra'], 'e')
(['enxertado', 'em', 'legítima', 'quanto'], 'oliveira')
(['que', 'pode', 'de', 'mim'], 'diante')


![](https://i.stack.imgur.com/P9jnI.png)

Temos uma rede neural rasa, o input $X$ é a média dos vetores *one hot* do contexto. A rede é descrita pelas seguintes equações: <br>
$$Z_1 = W_1X + b_1$$
$$h = Z_1$$
$$Z_2 = W_1h + b_2$$
$$ŷ = softmax(Z_2)$$
<br>
Em que a função softmax pode ser definida:
$$softmax(x_i) = \frac{e^{x_i}}{\sum_{k=1}^N e^{x_k}}$$

<img style="height:400px;" src = "https://mermaid.ink/img/eyJjb2RlIjoiZ3JhcGggVERcbiAgQVtJbnB1dF0gLS0-IE1NMSgqKVxuICBXMSAtLT4gTU0xXG4gIE1NMSAtLT4gWjFcbiAgYjEgLS0-IFoxXG4gIFoxIC0tPiB8UmVMVXxoXG4gIGggLS0-IE1NMigqKVxuICBXMiAtLT4gTU0yKCopXG4gIE1NMiAtLT4gWjJcbiAgYjIgLS0-IFoyXG4gIFoyIC0tPiB8c29mdG1heHzFt1xuXG4iLCJtZXJtYWlkIjp7InRoZW1lIjoiZGVmYXVsdCIsInRoZW1lVmFyaWFibGVzIjp7ImJhY2tncm91bmQiOiJ3aGl0ZSIsInByaW1hcnlDb2xvciI6IiNFQ0VDRkYiLCJzZWNvbmRhcnlDb2xvciI6IiNmZmZmZGUiLCJ0ZXJ0aWFyeUNvbG9yIjoiaHNsKDgwLCAxMDAlLCA5Ni4yNzQ1MDk4MDM5JSkiLCJwcmltYXJ5Qm9yZGVyQ29sb3IiOiJoc2woMjQwLCA2MCUsIDg2LjI3NDUwOTgwMzklKSIsInNlY29uZGFyeUJvcmRlckNvbG9yIjoiaHNsKDYwLCA2MCUsIDgzLjUyOTQxMTc2NDclKSIsInRlcnRpYXJ5Qm9yZGVyQ29sb3IiOiJoc2woODAsIDYwJSwgODYuMjc0NTA5ODAzOSUpIiwicHJpbWFyeVRleHRDb2xvciI6IiMxMzEzMDAiLCJzZWNvbmRhcnlUZXh0Q29sb3IiOiIjMDAwMDIxIiwidGVydGlhcnlUZXh0Q29sb3IiOiJyZ2IoOS41MDAwMDAwMDAxLCA5LjUwMDAwMDAwMDEsIDkuNTAwMDAwMDAwMSkiLCJsaW5lQ29sb3IiOiIjMzMzMzMzIiwidGV4dENvbG9yIjoiIzMzMyIsIm1haW5Ca2ciOiIjRUNFQ0ZGIiwic2Vjb25kQmtnIjoiI2ZmZmZkZSIsImJvcmRlcjEiOiIjOTM3MERCIiwiYm9yZGVyMiI6IiNhYWFhMzMiLCJhcnJvd2hlYWRDb2xvciI6IiMzMzMzMzMiLCJmb250RmFtaWx5IjoiXCJ0cmVidWNoZXQgbXNcIiwgdmVyZGFuYSwgYXJpYWwiLCJmb250U2l6ZSI6IjE2cHgiLCJsYWJlbEJhY2tncm91bmQiOiIjZThlOGU4Iiwibm9kZUJrZyI6IiNFQ0VDRkYiLCJub2RlQm9yZGVyIjoiIzkzNzBEQiIsImNsdXN0ZXJCa2ciOiIjZmZmZmRlIiwiY2x1c3RlckJvcmRlciI6IiNhYWFhMzMiLCJkZWZhdWx0TGlua0NvbG9yIjoiIzMzMzMzMyIsInRpdGxlQ29sb3IiOiIjMzMzIiwiZWRnZUxhYmVsQmFja2dyb3VuZCI6IiNlOGU4ZTgiLCJhY3RvckJvcmRlciI6ImhzbCgyNTkuNjI2MTY4MjI0MywgNTkuNzc2NTM2MzEyOCUsIDg3LjkwMTk2MDc4NDMlKSIsImFjdG9yQmtnIjoiI0VDRUNGRiIsImFjdG9yVGV4dENvbG9yIjoiYmxhY2siLCJhY3RvckxpbmVDb2xvciI6ImdyZXkiLCJzaWduYWxDb2xvciI6IiMzMzMiLCJzaWduYWxUZXh0Q29sb3IiOiIjMzMzIiwibGFiZWxCb3hCa2dDb2xvciI6IiNFQ0VDRkYiLCJsYWJlbEJveEJvcmRlckNvbG9yIjoiaHNsKDI1OS42MjYxNjgyMjQzLCA1OS43NzY1MzYzMTI4JSwgODcuOTAxOTYwNzg0MyUpIiwibGFiZWxUZXh0Q29sb3IiOiJibGFjayIsImxvb3BUZXh0Q29sb3IiOiJibGFjayIsIm5vdGVCb3JkZXJDb2xvciI6IiNhYWFhMzMiLCJub3RlQmtnQ29sb3IiOiIjZmZmNWFkIiwibm90ZVRleHRDb2xvciI6ImJsYWNrIiwiYWN0aXZhdGlvbkJvcmRlckNvbG9yIjoiIzY2NiIsImFjdGl2YXRpb25Ca2dDb2xvciI6IiNmNGY0ZjQiLCJzZXF1ZW5jZU51bWJlckNvbG9yIjoid2hpdGUiLCJzZWN0aW9uQmtnQ29sb3IiOiJyZ2JhKDEwMiwgMTAyLCAyNTUsIDAuNDkpIiwiYWx0U2VjdGlvbkJrZ0NvbG9yIjoid2hpdGUiLCJzZWN0aW9uQmtnQ29sb3IyIjoiI2ZmZjQwMCIsInRhc2tCb3JkZXJDb2xvciI6IiM1MzRmYmMiLCJ0YXNrQmtnQ29sb3IiOiIjOGE5MGRkIiwidGFza1RleHRMaWdodENvbG9yIjoid2hpdGUiLCJ0YXNrVGV4dENvbG9yIjoid2hpdGUiLCJ0YXNrVGV4dERhcmtDb2xvciI6ImJsYWNrIiwidGFza1RleHRPdXRzaWRlQ29sb3IiOiJibGFjayIsInRhc2tUZXh0Q2xpY2thYmxlQ29sb3IiOiIjMDAzMTYzIiwiYWN0aXZlVGFza0JvcmRlckNvbG9yIjoiIzUzNGZiYyIsImFjdGl2ZVRhc2tCa2dDb2xvciI6IiNiZmM3ZmYiLCJncmlkQ29sb3IiOiJsaWdodGdyZXkiLCJkb25lVGFza0JrZ0NvbG9yIjoibGlnaHRncmV5IiwiZG9uZVRhc2tCb3JkZXJDb2xvciI6ImdyZXkiLCJjcml0Qm9yZGVyQ29sb3IiOiIjZmY4ODg4IiwiY3JpdEJrZ0NvbG9yIjoicmVkIiwidG9kYXlMaW5lQ29sb3IiOiJyZWQiLCJsYWJlbENvbG9yIjoiYmxhY2siLCJlcnJvckJrZ0NvbG9yIjoiIzU1MjIyMiIsImVycm9yVGV4dENvbG9yIjoiIzU1MjIyMiIsImNsYXNzVGV4dCI6IiMxMzEzMDAiLCJmaWxsVHlwZTAiOiIjRUNFQ0ZGIiwiZmlsbFR5cGUxIjoiI2ZmZmZkZSIsImZpbGxUeXBlMiI6ImhzbCgzMDQsIDEwMCUsIDk2LjI3NDUwOTgwMzklKSIsImZpbGxUeXBlMyI6ImhzbCgxMjQsIDEwMCUsIDkzLjUyOTQxMTc2NDclKSIsImZpbGxUeXBlNCI6ImhzbCgxNzYsIDEwMCUsIDk2LjI3NDUwOTgwMzklKSIsImZpbGxUeXBlNSI6ImhzbCgtNCwgMTAwJSwgOTMuNTI5NDExNzY0NyUpIiwiZmlsbFR5cGU2IjoiaHNsKDgsIDEwMCUsIDk2LjI3NDUwOTgwMzklKSIsImZpbGxUeXBlNyI6ImhzbCgxODgsIDEwMCUsIDkzLjUyOTQxMTc2NDclKSJ9fSwidXBkYXRlRWRpdG9yIjpmYWxzZX0/">

Nossa função de custo é a entropia cruzada, que é definida da seguinte forma:
$$J = - \sum_{k=1}^V y_k \ln\hat{y}_k$$
Em que V denota o número de palavras no vocabulário. <br>

Perceba que essa função de custo faz sentido para o nosso problema, pois quando $y_k$ é igual a zero, independetemente do valor de $\hat{y}_k$, a multiplicação será zero. O resultado do somatório será apenas o $-\ln \hat{y}_k$ para o $\hat{y}_k$ correspondente ao $k$ da classe correta.

 - Se $\hat{y}_k$ se aproxima de 1 (o que nós queremos, pois isso representa a probabilidade prevista para o modelo para a classe correta) $J$ se aproxima de 0; 
 - Se $\hat{y}_k$ se aproximar de 0, $J$ explode para o infinito (já que $\ln x$ vai para - infinito quando x se aproxima de 0)<br> 

Queremos minimizar a nossa função de custo. Para isso, vamos aplicar o gradiente descendente nos nossos parâmetros aprendidos $W_1$, $W_2$, $b_1$ e $b_2$. Isso será feito utilizando backpropagation. 
As expressões das derivadas não devem ser difíceis de se obter com um conhecimento básico de cálculo diferencial, com exceção da derivação $\frac{\partial J}{\partial Z_2}$, ela pode ser entendida [neste vídeo](https://www.youtube.com/watch?v=5-rVLSc2XdE). <br>

As expressões são (para um batch com $m$ elementos):
$$\frac{\partial J_b}{\partial W_2} = \frac{1}{m}(\hat{y} - y)h^t$$

$$\frac{\partial J_b}{\partial W_1} = \frac{1}{m} \cdot W_2^t(\hat{y} - y)$$

<br>
E, como praxe do gradiente descendente, os parâmtros serão atualizados ($\alpha$ é a taxa de aprendizado):
$$W_2 := W_2 - \alpha \cdot \frac{\partial J_b}{\partial W_2}$$
$$W_1 := W_1 - \alpha \cdot \frac{\partial J_b}{\partial W_1}$$

### Implementação com Numpy

In [16]:
def word_to_one_hot_vector(word, vocab, V):
    one_hot_vector = np.zeros(V)
    one_hot_vector[vocab[word]] = 1
    return one_hot_vector

In [17]:
def get_batch(raw_batch, vocab, V, batch_size):
    batch_x = []
    batch_y = []
    for x, y in raw_batch:
        while len(batch_x) < batch_size:
            batch_x.append(np.mean([word_to_one_hot_vector(w, vocab, V) for w in x], axis=0))
            batch_y.append(word_to_one_hot_vector(y, vocab, V))
        else:
            return np.array(batch_x).T, np.array(batch_y).T

In [18]:
def initialize_params(V,emb_size):
    W1 = np.random.rand(emb_size,V)
    W2 = np.random.rand(V,emb_size)
    
    return W1, W2

In [19]:
def softmax(z):
    e_z = np.exp(z)
    sum_e_z = np.sum(e_z, axis=0) 
    soft = e_z / sum_e_z
    return soft

In [45]:
def foward(x, W1, W2):
    z1 = W1@x
    h = z1
    z2 = W2@h
    yhat = softmax(z2)
    return yhat, h

In [21]:
def compute_cost(y, yhat, batch_size):
    logprobs = y*np.log(yhat)   
    cost = - (1/batch_size) * np.sum(logprobs)
    cost = np.squeeze(cost)
    return cost

In [55]:
def back_prop(x, yhat, y, h, W1, W2, batch_size):
    grad_W1 = (1/batch_size)*np.dot(np.dot(W2.T,(yhat-y)),x.T)
    grad_W2 = (1/batch_size)*np.dot((yhat - y),h.T)
    return grad_W1, grad_W2

In [57]:
BATCH_SIZE = 256
EPOCHS = 1
ALPHA = 0.001
emb_size = 50


W1, W2 = initialize_params(V,emb_size)
for i in range(EPOCHS):
    for j in range(int(len(dataset)/BATCH_SIZE)):
        x,y = get_batch(dataset, vocab, V, BATCH_SIZE)
        yhat, h = foward(x, W1, W2)
        cost = compute_cost(y, yhat, BATCH_SIZE)
        if j%200 == 0:
            print("iteration: {:4.0f} | epoch {:4.0f} | cost: {:.3f}".format(j,i,cost))
        grad_W1, grad_W2 = back_prop(x, yhat, y, h, W1, W2, BATCH_SIZE)
        W1 -= ALPHA*grad_W1
        W2-= ALPHA*grad_W2

iteration:    0 | epoch    0 | cost: 10.455
iteration:  200 | epoch    0 | cost: 7.504
iteration:  400 | epoch    0 | cost: 4.494
iteration:  600 | epoch    0 | cost: 1.680
iteration:  800 | epoch    0 | cost: 0.463
iteration: 1000 | epoch    0 | cost: 0.206
iteration: 1200 | epoch    0 | cost: 0.125
iteration: 1400 | epoch    0 | cost: 0.088
iteration: 1600 | epoch    0 | cost: 0.067
iteration: 1800 | epoch    0 | cost: 0.054
iteration: 2000 | epoch    0 | cost: 0.045
iteration: 2200 | epoch    0 | cost: 0.039


In [58]:
embeddings = (W1.T + W2)

In [59]:
encoded_embeddings = {}
for word, i in vocab.items():
    encoded_embeddings[word] = list(embeddings[i])

In [60]:
import json
with open(f'bible_embeddings.json', 'w') as file:
    file.write(json.dumps(encoded_embeddings)) 

In [61]:
with open('bible_embeddings.json') as json_file:
    emb = json.load(json_file)

In [62]:
def cossine_similarity(x,y):
    return np.dot(x.T,y)/(np.linalg.norm(x)*np.linalg.norm(y))

def find_most_similar(x, word_emb):
    similar = {}
    for word, vec in word_emb.items():
        sim = cossine_similarity(np.array(word_emb[x]),np.array(vec))
        if word != x:
            similar[word] = sim
    similar =  {k: v for k, v in sorted(similar.items(), key=lambda item: item[1], reverse=True)}
    most_10_similar = list(similar.keys())[:10]
    most_similar = [(x, similar[x]) for x in most_10_similar]
    return most_similar

In [64]:
find_most_similar('rei', emb)

[('assolarem', 0.9473805174572902),
 ('sustentar', 0.942844763089323),
 ('apaguem', 0.9418055271555023),
 ('necessitado', 0.9403377856324923),
 ('zombará', 0.9396796340451484),
 ('vangloriosos', 0.9396168729618221),
 ('pousares', 0.9384967324334551),
 ('pele', 0.9383287819873256),
 ('deixareis', 0.9377618242510127),
 ('maalalel', 0.937494105608405)]

In [63]:
find_most_similar('jesus', emb)

[('atentarem', 0.9362621218135676),
 ('peludas', 0.9314442961005218),
 ('mesquinho', 0.927347094668162),
 ('obrigado', 0.9270888131980677),
 ('lamentam', 0.9255509062051331),
 ('suscitará', 0.9245826464904654),
 ('trilhada', 0.9245418495871436),
 ('durarão', 0.9244502438394895),
 ('rejeitais', 0.9243248507055914),
 ('regalava', 0.9242017583090081)]

In [17]:
def analogy(x, a, b, word_emb):
    reference = np.array(word_emb[a]) - np.array(word_emb[b])
    analogy_score = -999
    analogy_word = None
    
    for word, vec in word_emb.items():
        analogy_vec = np.array(word_emb[x]) - np.array(vec)
        sim = cossine_similarity(reference,analogy_vec)
        if sim>analogy_score:
            analogy_score = sim
            analogy_word = word
            
    return analogy_word, analogy_score